In [6]:
!pip install pandas==1.0.3
!pip install tensorflow==2.0

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv
/kaggle/input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv
/kaggle/input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths_US.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed_US.csv


#Data Cleaning and Processing

In [8]:
#read the latest csv value
df =pd.read_csv("/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv")

In [ ]:
pd.__version__

In [9]:
df.head()

ImportError: cannot import name 'is_url'

   SNo ObservationDate Province/State  Country/Region      Last Update  \
0    1      01/22/2020          Anhui  Mainland China  1/22/2020 17:00   
1    2      01/22/2020        Beijing  Mainland China  1/22/2020 17:00   
2    3      01/22/2020      Chongqing  Mainland China  1/22/2020 17:00   
3    4      01/22/2020         Fujian  Mainland China  1/22/2020 17:00   
4    5      01/22/2020          Gansu  Mainland China  1/22/2020 17:00   

   Confirmed  Deaths  Recovered  
0        1.0     0.0        0.0  
1       14.0     0.0        0.0  
2        6.0     0.0        0.0  
3        1.0     0.0        0.0  
4        0.0     0.0        0.0  

In [ ]:
#select only Germany as country
filt_germany = (df['Country/Region'] == 'Germany') 

In [ ]:
#overwrite dataframe : only want to work with India Case
df = df[filt_germany]
df

In [ ]:
#removing duplicate rows if any
df.drop_duplicates(inplace=True)

In [ ]:
# dropping State because its NaN and Country since its fixed value : Germany
df.drop(columns=['Province/State', 'Country/Region'], inplace=True)

In [ ]:
#dropping Last Update column since it won't help in forecasting
df.drop(columns='Last Update', inplace=True)

In [ ]:
#set SNo as index
df.set_index('SNo', inplace=True)

In [ ]:
#convert Object type of ObservationDate to DateTime
df['ObservationDate'] = pd.to_datetime(df['ObservationDate'])

In [ ]:
#Sorting the sequence based on the observation date
df.sort_values('ObservationDate', inplace=True)

In [ ]:
df.columns

In [ ]:
#visualize observation date with confirmed cases
data = df['Confirmed']
data.index = df['ObservationDate']
data.plot()

In [ ]:
data = data.values

In [ ]:
data = data.astype(np.float32)

In [ ]:
data

In [ ]:
def create_sequence(input_data, steps):
    i = 0
    x = []
    y = []
    while (i+steps) < len(input_data):
        x.append(input_data[i:i+steps])
        y.append(input_data[i+steps])
        i = i + 1
    return x, y

In [ ]:
# test the create_sequence
input_data = [10, 20, 30, 40, 50, 60]
x,y = create_sequence(input_data, 3)
print(x)
print(y)

In [ ]:
n_steps = 3
x, y = create_sequence(data, n_steps)

In [ ]:
x = np.asarray(x)
y = np.asarray(y)

In [ ]:
row_index = x.shape[0] - 1

In [ ]:
x, x_test = x[:row_index], x[row_index]
y, y_test = y[:row_index], y[row_index]

In [ ]:
print(x.shape)

In [ ]:
print(y, y_test)

In [ ]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
x = x.reshape((x.shape[0], x.shape[1], n_features))
print(x.shape)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM
tf.random.set_seed(100)

In [ ]:
print(n_steps, n_features)

In [ ]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu',return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
model.fit(x, y, epochs=1000, verbose=2)

In [ ]:
x_test = x_test.reshape((1, n_steps, n_features))
yhat = model.predict(x_test, verbose=0)

In [ ]:
x_test

In [ ]:
print(f"Predicted Value: {yhat[0][0]}")
print(f"Original value: {y_test}")

In [ ]:
model.save("germany_prediction_05_april.h5")